# Deploy ML models to SageMaker with MLFlow 

We have gone through a lot of different solutions to put our machine learning model into production. But we haven't yet explored many of the solutions available today: the _cloud_.

Cloud solutions can represent a perfect fit for you needs. Most of them are easy to set up, as you will realize soon enough, but there are not free of charge. You should not neglect this factor when thinking about your deployment solution.

Otherwise, cloud solutions offer you a lot of really useful features. In this chapter, you will understand how to scale your models using cloud services as well as monitor them. Let's get started! ✊

## What you will learn in this course

Serving models locally is fine but they won't be available once you will turn your computer off. In this course, you will learn to push your ML model into development on <a href="https://aws.amazon.com/sagemaker/" target="_blank">AWS Sagemaker</a>.

This lecture is going to re-use some knowledge you acquired during the previous days. Here are the important steps we are going through:

- ⓵ Setup our environment using <a href="https://docs.conda.io/en/latest/" target="_blank">Conda</a> and some CLI tools
- ⓶ Create what we need in AWS
- ⓷ Training and logging our model with MLFlow
- ⓸ Pushing our model into production on Sagemaker
- ⓹ Requesting our model

## ⓵ Prepare your environment with Conda

We already have seen how to create a virtual environment using `virtualenv`. But it isn't the only tool to build and manage your environment. You can use Conda to do so. We assume you have <a href="https://docs.anaconda.com/anaconda/install/" target="_blank">Anaconda</a> already setup on your machine.

### Start a new environment

In your terminal (MacOS, Linux) or Anaconda Powershell Prompt (Windows) create a new environment for this project:

```shell
$ conda create -n mlflow
```

You can name it as you want. Here we named it `mlflow`. To activate it, enter:

```shell
$ conda activate mlflow
```

You should see `(mlflow)` at the line beginning of your line.

### Setup MLFlow

All those steps should be done inside your activated conda environment (we name ours: `mlflow`).

#### On Windows

As for now, if you want to be able to execute `mlflow ui` you need to follow those steps. In your Anaconda Powershell Prompt, we first install all dependencies using conda:

```shell
$ conda install -c conda-forge mlflow --only-deps
```

Once it finishes, we setup MLFlow with `pip`:

```shell
$ pip install mlflow
```

And you are done!

#### On MacOS or Linux

Installing on MacOS or Linux is pretty straightforward and should not be a problem for you:

```shell
$ conda install -c conda-forge mlflow
```

Accept and you should be good to go!

### Check MLFlow

You can check if everything wen't well:

```shell
$ mlflow --version
mlflow, version 1.12.0
```

### Installing other dependencies

We are going to need `sklearn` and `boto3`. While your environment is activated:

```shell
$ pip install sklearn boto3
```

> If you need to install a library in your environment you can install it either with `conda install` or with the good ol' `pip install`. Sometimes `conda` brings more packages and less overhead with installment than `pip`.

### Install AWS CLI 

You need to install <a href="https://aws.amazon.com/cli/" target="_blank">AWS CLI</a> to setup a region and default credential. To install AWS CLI, follow one of these links: 

- <a href="https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2-linux.html" target="_blank">AWS CLI Linux</a>
- <a href="https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2-mac.html" target="_blank">AWS CLI for Mac</a>
- <a href="https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2-windows.html" target="_blank">AWS CLI for Windows</a>

Once you have it installed, you can go to your terminal and run:

```shell 
$ aws --version
```

You should get the following output: 

```shell 
aws-cli/2.0.22 Python/3.7.4 Darwin/19.5.0 botocore/2.0.0dev26
```

### Setup AWS creds & region 

For MLFlow to work, you will need to setup AWS credentials and region. Here are the commands: 

```shell
$ aws configure set aws_access_key_id ACCESS_KEY
$ aws configure set aws_secret_access_key SECRET_KEY
$ aws configure set default.region REGION
```

Replace with your credentials and the region you want your model to be deployed (we use `eu-west-3`).

An we are done setting up our environment and tools!

## ⓶ Configuring AWS

### Create an execution role 

If you want a working deployment, you will need to create a role with _AMAZON_SAGEMAKER_FULL_ACCESS_.

<img src="https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/ecr_execution_role.png" alt="IAM role" width=700>

Copy the ARN, and keep it aside. We will need to use it for deployment later on.

## ⓷ Training and logging our model with MLFlow

We have already coded the training process for you. It should not be too unfamiliar to you so we won't go in further explanations:

In [ ]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

iris = load_iris()

X = pd.DataFrame(data = iris["data"], columns= iris["feature_names"])
y = pd.DataFrame(data = iris["target"], columns=["target"])

X_train, X_test, y_train, y_test = train_test_split(X, y)

with mlflow.start_run():
    # Specified Parameters 
    c = 0.5

    # Instanciate and fit the model 
    lr = LogisticRegression(C=c)
    lr.fit(X_train.values, y_train.values)

    # Store metrics 
    predicted_qualities = lr.predict(X_test.values)
    accuracy = lr.score(X_test.values, y_test.values)

    # Print results 
    print("LogisticRegression model")
    print("Accuracy: {}".format(accuracy))

    # Log Metric 
    mlflow.log_metric("Accuracy", accuracy)

    # Log Param
    mlflow.log_param("C", c)
    
    # Log model
    mlflow.sklearn.log_model(lr, "model")

Start MLFlow UI, you should see at least one line corresponding to this new model, click on it, you access a page with the run ID on the top (which we are going to use in few steps):

<img src="https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/FULL_STACK_12_WEEK/M09/M09-D05-00-01-Run_id.png" alt="Run ID" width=700>

## ⓸ Deploy to SageMaker 

### Pushing the Docker environment

Before pushing our model into production we need to create the execution environment. Hopefully for us, we don't have to hassle because MLFlow do the job for us.

Enter the following line in you terminal:

```shell
$ mlflow sagemaker build-and-push-container
```

Wait until it finish by pushing the Docker image to ECR. It can take a while so take a break! ☕️

### Get ECR ARN

Now go to <a href="https://aws.amazon.com/" target="_blank">AWS</a> and click on the new registry created by MLFLow – it should be something like _mlflow-pyfunc_.

<img src="https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/FULL_STACK_12_WEEK/M09/M09-D05-00-01-ECR_registry.png" alt="ECR View" width=700>

Then copy the URI of our Docker image:

<img src="https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/FULL_STACK_12_WEEK/M09/M09-D05-00-01-Copy_URI.png" alt="Copy URI" width=700>

You can fill the variable `image_ecr_url` with this Docker image URI:

In [ ]:
app_name = "iris"

# Region where you want your model to be deployed
region = "eu-west-3"

# You can change the instance type
instance_type = "ml.t2.medium"

# Get it from Mlflow
run_id = ""
model_uri = "runs:/" + run_id + "/model"

# Put your AWS Role ARN
execution_role = "arn:aws:iam::802836651515:role/SagemakerFullAccess"

# Put the image URI took from AWS ECR
image_ecr_url = "802836651515.dkr.ecr.eu-west-3.amazonaws.com/mlflow-pyfunc:1.13.1"

Be sure to fill the last cell with:

- `run_id`: the run ID from MLFlow UI,
- `execution_role`: the ARN of our IAM role we created before,
- `image_ecr_url`: the URI we copied just now.

We set the most basic instance type. You can change for <a href="https://aws.amazon.com/releasenotes/sagemaker-instance-types-in-paris-eu-west-3/" target="_blank">better instances</a> if you need to.

> Check the <a href="https://aws.amazon.com/fr/free/?all-free-tier.sort-by=item.additionalFields.SortRank&all-free-tier.sort-order=asc" target="_blank">AWS Free tier</a>, if you want to make you won't be charged for using instances. 

Finally, we can run the following command to deploy your ML model to SageMaker:

In [ ]:
# Create a SageMaker app 
import mlflow.sagemaker as mfs

mfs.deploy(
    app_name=app_name,
    model_uri=model_uri,
    image_url=image_ecr_url,
    region_name=region,
    mode="replace",
    execution_role_arn=execution_role,
    instance_type=instance_type
)

## ⓹ Use your deployed model 

Now you can use your deploy model. You first need to prepare your data the following way:

In [ ]:
# Format input for API 
input_json = X_test.to_json(orient="split")

And let's create the following function: 

In [4]:
# Use the API endpoint
import json
import boto3

def query_endpoint(app_name, input_json):
    # Instanciate a client in order to access Sagemaker
    client = boto3.session.Session().client("sagemaker-runtime", region)
    # Call our Iris endpoint
    response = client.invoke_endpoint(
        EndpointName=app_name,
        Body=input_json,
        ContentType='application/json; format=pandas-split',
    )
    # We get the answer
    preds = response['Body'].read().decode("ascii")
    preds = json.loads(preds)
    print("Received response: {}".format(preds))

    return preds

Use the function: 

In [3]:
# Evaluate the input by posting it to the deployed model
prediction1 = query_endpoint(app_name=app_name, input_json=input_json)

NameError: name 'input_json' is not defined

## Delete the app 

Delete your app when you have finished:

In [ ]:
# Delete app
import mlflow.sagemaker as mfs

app_name = "iris"

# Specify the archive=False option to delete any SageMaker models and configurations
# associated with the specified application
mfs.delete(app_name=app_name, region_name=region, archive=False)

Once you are done, you should clear also the ECR and your S3 (MLFlow use it to upload your model).

## Resources 

- <a href="https://zestedesavoir.com/tutoriels/1448/installer-un-environnement-de-developpement-python-avec-conda/" target="_blank">🇫🇷 Installer un environnement avec Conda</a>
* <a href="https://docs.databricks.com/administration-guide/cloud-configurations/aws/sagemaker.html" target="_blank">Set up AWS authentication for SageMaker deployment</a>
* <a href="https://docs.databricks.com/_static/notebooks/mlflow/mlflow-quick-start-deployment-aws.html" target="_blank">MLflow Quick Start Part 2: Serving Models via Amazon SageMaker</a>
* <a href="https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html" target="_blank">Installing the AWS CLI version 2</a>
* <a href="https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2-windows.html" target="_blank">Installing the AWS CLI version 2 on Windows</a>